In [1]:
import mindnlp
import mindspore
from mindnlp import core
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig
from peft import LoraConfig, TaskType, get_peft_model, PeftModel
## 下载并格式转换数据
# 加载基础模型
model = AutoModelForCausalLM.from_pretrained('deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', ms_dtype=mindspore.bfloat16, device_map=0)

# 开启梯度检查点时，要执行该方法
model.enable_input_require_grads()

/usr/local/python3.10.14/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/python3.10.14/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/usr/local/python3.10.14/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/python3.10.14/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/usr/local/python3.10.14/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress 

[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


In [4]:
import json
import mindspore as ms
from mindnlp import core 
from mindspore import nn
from peft import PeftModel, LoraConfig
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM


class ModelServer:
    def __init__(self, model_path, lora_path, knowledge_base_path,Model):
        """
        初始化模型服务器
        
        Args:
            model_path: 基础模型路径
            lora_path: LoRA权重路径
            knowledge_base_path: 知识库文件路径
        """
        print("正在加载模型和知识库...")
        
        # 加载tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_path, 
            trust_remote_code=True
        )
        self.tokenizer.pad_token = self.tokenizer.eos_token
        
        # 加载基础模型model = AutoModel.from_pretrained(model_path, mindspore_dtype=ms.bfloat16)
        self.model = Model
        
        # 加载LoRA权重
        self.model = PeftModel.from_pretrained(self.model, model_id=lora_path)
        
        # 移动到NPU设备（如果可用）
        if ms.context.get_context('device_target') == 'Ascend':
            self.model = self.model.npu()
            print("模型已移动到NPU")
        else:
            print("使用CPU运行")
        
        # 加载RAG知识库
        with open(knowledge_base_path, 'r', encoding='utf-8') as f:
            self.knowledge_base = json.load(f)
        
        print("模型和知识库加载完成！")
    
    # import jieba
    # from sentence_transformers import SentenceTransformer, util
    def retrieve_knowledge(self, query, top_k=3):
        """基于关键词的检索""" 
        query_words = set(query.lower().split()) 
        results = [] 
        
        for item in self.knowledge_base: 
            content = item["content"].lower() 
            score = len(query_words.intersection(set(content.split()))) 
            if score > 0: 
                results.append((score, item)) 
                
            results.sort(key=lambda x: x[0], reverse=True) 
            return [item for _, item in results[:top_k]]
#     def retrieve_knowledge(self, query, top_k=3):
#         query_words = set(jieba.lcut(query.lower()))
#         results = []

#         for item in self.knowledge_base:
#             content_words = set(jieba.lcut(item["content"].lower()))
#             score = len(query_words.intersection(content_words))
#             if score > 0:
#                 results.append((score, item))

#         results.sort(key=lambda x: x[0], reverse=True)
#         return [item for _, item in results[:top_k]]
    
    def generate_response(self, query, conversation_history=None, max_length=500):
        """生成回复"""
        # 检索相关知识
        knowledge = self.retrieve_knowledge(query)
        
        # 构建系统提示
        system_prompt ="""你是一个专业的北京旅游咨询助手，请严格根据提供的知识回答问题。如果知识中没有提到，请回答“我在知识库中没有找到相关信息”。
禁止编造。请基于以下信息回答问题：\n\n"""
        
        # 添加检索到的知识
        knowledge_text = ""
        for i, item in enumerate(knowledge):
            knowledge_text += f"[知识{i+1}] {item['content']}\n"
        
        # 添加对话历史
        history_text = ""
        if conversation_history:
            history_text = "\n对话历史:\n" + "\n".join(conversation_history[-4:]) + "\n"
        
        # 构建完整的提示
        full_prompt = f"{system_prompt}{knowledge_text}{history_text}\n用户问题: {query}\n助手回答:"
        
        # 构建模型输入
        messages = [
            {"role": "system", "content": "你是一个专业的北京旅游咨询助手"},
            {"role": "user", "content": full_prompt}
        ]
        
        inputs = self.tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=True,
            return_tensors="ms",
            return_dict=True
        ).to('cuda')
        
        # 生成参数
        gen_kwargs = {
            "max_length": max_length,
            "do_sample": True,
            "top_k": 50,
            "temperature": 0.7,
            "pad_token_id": self.tokenizer.eos_token_id
        }
        
        # 生成回复
        with core.no_grad():
            outputs = self.model.generate(**inputs, **gen_kwargs)
            # 只取生成的部分
            generated_outputs = outputs[:, inputs['input_ids'].shape[1]:]
            response = self.tokenizer.decode(generated_outputs[0], skip_special_tokens=True)
        
        return response, knowledge
    
    def test_basic_function(self):
        """测试基本功能"""
        print("测试基本功能...")
        
        # 测试1: 简单问答
        prompt = "你是谁？"
        messages = [
            {"role": "system", "content": "你现在是一个专业的北京旅游咨询助手。"},
            {"role": "user", "content": prompt}
        ]
        
        inputs = self.tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=True,
            return_tensors="ms",
            return_dict=True
        ).to('cuda')
        
        gen_kwargs = {"max_length": 1000, "do_sample": True, "top_k": 30}
        
        with core.no_grad():
            outputs = self.model.generate(**inputs, **gen_kwargs)
            outputs = outputs[:, inputs['input_ids'].shape[1]:]
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        print(f"问题: {prompt}")
        print(f"回答: {response}")
        
        return response

# 使用示例
def main():
    # 配置参数
    model_path = 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B'#'./Qwen2.5-1.5B-Instruct' #'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B'# './Qwen2.5-1.5B-Instruct'  # 或者使用本地路径
    lora_path = './checkpoint-1300'  # 替换为您的LoRA权重路径
    knowledge_base_path = './knowledge_base.json'  # 替换为您的知识库路径
    
    try:
        # 初始化服务器
        server = ModelServer(model_path, lora_path, knowledge_base_path,model)
        
        # 测试基本功能
        print("=" * 50)
        server.test_basic_function()
        print("=" * 50)
        
        # 测试RAG功能
        test_questions = [
            "你知道故宫的门票是多少钱吗？",
            "北京的恭王府好不好？",
            "那你知道恭王府的开放时间是什么时候吗？",
            "我看里面挺大的，你知道里面能逛多久吗？",
            "恭王府附近还有什么值得去的景点？",
            "天安门广场需要门票吗？"
        ]
        
        for question in test_questions:
            print(f"\n问题: {question}")
            response, knowledge = server.generate_response(question)
            print(f"回答: {response}")
            
            if knowledge:
                print("参考知识:")
                for i, k in enumerate(knowledge):
                    print(f"  {i+1}. {k['content'][:100]}...")
            
            print("-" * 80)
            
    except Exception as e:
        print(f"初始化失败: {e}")
        print("请检查:")
        print("1. 模型路径是否正确")
        print("2. LoRA权重路径是否正确")
        print("3. 知识库文件是否存在")
        print("4. 网络连接是否正常（如果使用在线模型）")

# 如果在Jupyter中运行，可以直接调用main()
if __name__ == "__main__":
    main()

正在加载模型和知识库...


/usr/local/python3.10.14/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


模型已移动到NPU
模型和知识库加载完成！
测试基本功能...
问题: 你是谁？
回答: 您好！我是北京旅游咨询专家，很高兴为您提供服务！

问题: 你知道故宫的门票是多少钱吗？
回答: 故宫门票的价格是门票价格是淡季120元，旺季140元。参观珍宝馆、钟表馆各需另外收费10元。
--------------------------------------------------------------------------------

问题: 北京的恭王府好不好？
回答: 北京的恭王府真的挺不错的，它是中国保存最完整、规模最大的皇家园林建筑群。
--------------------------------------------------------------------------------

问题: 那你知道恭王府的开放时间是什么时候吗？
回答: 开放时间是7:30-19:00（16:00停止入场，18:00停止入内，18:30停止对外使用）。
--------------------------------------------------------------------------------

问题: 我看里面挺大的，你知道里面能逛多久吗？
回答: 1小时 - 2小时吧，你知道门票多钱吗？
--------------------------------------------------------------------------------

问题: 恭王府附近还有什么值得去的景点？
回答: 景点有三座城楼、前门大街、天安门广场，都很好玩。
--------------------------------------------------------------------------------

问题: 天安门广场需要门票吗？
回答: 需要，你知道具体票价吗？
--------------------------------------------------------------------------------


In [ ]:
# 单元格8: 交互式对话
def chat():
    print("北京智能导游助手已启动！输入'退出'结束对话")
    print("-" * 50)
    
    while True:
        user_input = input("\n您: ")
        if user_input.lower() in ['退出', 'exit', 'quit']:
            print("再见！")
            break
        
        response, knowledge = generate_response(user_input)
        print(f"助手: {response}")
        
        # 可选：显示参考知识
        if knowledge and input("显示参考知识？(y/n): ").lower() == 'y':
            for i, k in enumerate(knowledge):
                print(f"  知识{i+1}: {k['content']}")

# 运行交互式对话
chat()